## Utility functions

In [24]:
import os
import numpy as np
import pandas as pd
import time
from scipy import stats
import scipy
from statsmodels.regression.linear_model import yule_walker
import multiprocessing
from statsmodels.tsa.ar_model import AR

import warnings
warnings.filterwarnings('ignore')

#### gibbs sampler with parallel computing, diagnosis, etc.
# N is the number of iterations
# R is the indicator matrix of observations, dim(R)=(n,m)
# Y is the ratings (binary), dim(Y)=(n,m)
# W is the user feature matrix, dim(W)=(n,J)
# X is the item feature matrix, dim(X)=(m,K)
# Z is the initialized indicators
# U is the initialized noise matrix, dim(U)=(n,L)
# V is the initialized noise matrix, dim(V)=(m,L)
# L is the dimension of U_i and V_j
# alpha_y, beta_y, sigma_u2, sigma_v2 are initial values from prior
# alp_u, beta_u, alp_v, beta_v, sig_y2 are prior parameters (we choose sig_y2 large for a flat prior)
# N_tot is the total number of iterations
# N_sing is the number of iterations for each check
# n_chain is the number of markov chains
def full_gibbs(N_tot, N_sing, n_chain, R, Y, W, X, Z, L, alp_u, beta_u, alp_v, beta_v, sig_y2, a_r, b_r):
    pool = multiprocessing.Pool(processes=n_chain)
    
    # initiate
    args = []
    for i in range(n_chain):
        init = initial_gibbs(Y,W,X,Z,R,L,sig_y2, alp_u, beta_u, alp_v, beta_v, a_r, b_r)
        new_arg = list((N_sing,R,Y,W,X) + init + (alp_u, beta_u, alp_v, beta_v, sig_y2))
        args.append(new_arg)
    results = pool.map(base_gibbs_wrapper, args)

#### initialize samples for a gibbs sampler chain
def initial_gibbs(Y,W,X,Z,R,L, sig_y2, alp_u, beta_u, alp_v, beta_v, a_r, b_r):
    n, m = R.shape
    J = W.shape[1]
    K = X.shape[1]
    
    # sample from prior to start ~ start from estimator
    alpha_y = np.random.normal(0, np.sqrt(sig_y2), size = (1,K)) # alpha_y ~ N(0,sigma_y^2 I_K)
    beta_y = np.random.normal(0, np.sqrt(sig_y2), size = (1,J)) # beta_y ~ N(0,sigma_y^2 I_J)
    sigma_u2 = stats.invgamma.rvs(a= alp_u, scale = beta_u, size=1) # sigma_u^2 ~ IG(alp_u, beta_u)
    sigma_v2 = stats.invgamma.rvs(a= alp_v, scale = beta_v, size=1) # sigma_v^2 ~ IG(alp_v, beta_v)
    p_r = np.random.beta(a_r, b_r, size=1)
   
    
    # sample initial from kinds of posterior / expectation
    U = np.random.normal(0,1, (n, L)) # sample iid N(0,1)
    V = np.random.normal(0,1, (m, L)) # sample iid N(0,1)
    Z = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            if R[i,j] == 1 and Y[i,j] == 0:
                Z[i,j] = -np.sqrt(2/np.pi)
            elif R[i,j] ==1 and Y[i,j] == 1:
                Z[i,j] = np.sqrt(2/np.pi)
    
    # sample posterior in gibbs sampler
    a_p = a_r + sum(sum(R))
    b_p = b_r + sum(sum(1-R))
    p_e = np.random.beta(a_p, b_p, size=1)
    
    return( (Z, U, V, alpha_y, beta_y, sigma_u2, sigma_v2) )
    
def base_gibbs_wrapper(args):
    return base_gibbs(*args)

#### the basic gibbs sampler
# N is the number of iterations
# R is the indicator matrix of observations, dim(R)=(n,m)
# Y is the ratings (binary), dim(Y)=(n,m)
# W is the user feature matrix, dim(W)=(n,J)
# X is the item feature matrix, dim(X)=(m,K)
# Z is the initialized indicators
# U is the initialized noise matrix, dim(U)=(n,L)
# V is the initialized noise matrix, dim(V)=(m,L)
# L is the dimension of U_i and V_j
# alpha_y, beta_y, sigma_u2, sigma_v2 are initial values from prior
# alp_u, beta_u, alp_v, beta_v, sig_y2 are prior parameters (we choose sig_y2 large for a flat prior)
def base_gibbs(N, R, Y, W, X, Z, U, V, alpha_y, beta_y, sigma_u2, sigma_v2, alp_u, beta_u, alp_v, beta_v, sig_y2):
    n, m = R.shape
    J = W.shape[1]
    K = X.shape[1]
    L = U.shape[1]
    
    SIG_U2 = []
    SIG_V2 = []
    ALP_Y = []
    BETA_Y = []
    UU = []
    VV = []
    ZZ = []
    TIMES = []
    LOG_LIKES = []
    
    OBi = []
    for i in range(n):
        observed = [j for j in range(m) if R[i,j]==1.0]
        OBi.append(observed)
    OBj = []
    for j in range(m):
        observed = [i for i in range(n) if R[i,j]==1.0]
        OBj.append(observed)
    
    start_time = time.time()
    
    for nn in range(N):
        if nn%100 == 0:
            now_time = time.time()
            TIMES.append(now_time - start_time)
            start_time = time.time()
        
        #print(nn)
        # sample sigma_u2 and sigma_v2
        sigma_u2 = stats.invgamma.rvs(a= alp_u + n*L/2, scale = beta_u + sum(sum(U*U))/2, size=1)
        sigma_v2 = stats.invgamma.rvs(a= alp_v + m*L/2, scale = beta_v + sum(sum(V*V))/2, size=1)
        SIG_U2.append(sigma_u2[0])
        SIG_V2.append(sigma_v2[0])
        
        # sample alpha_y and beta_y
        sig_alp = np.eye(K)/sig_y2
        sum_r = sum(R)
        for j in range(m):
            sig_alp = sig_alp + sum_r[j] * np.matmul(np.matrix(X[j,]).transpose(), np.matrix(X[j,]))
        L_alp = np.linalg.cholesky(sig_alp)
        Linv_alp = scipy.linalg.solve_triangular(L_alp, np.eye(K), lower=True)
        sig_alp = np.matmul(Linv_alp.transpose(), Linv_alp) #choleskey decomp
        
        sig_beta = np.eye(J)/sig_y2
        sum_rt = sum(R.transpose())
        for i in range(n):
            sig_beta = sig_beta + sum_rt[i] * np.matmul(np.matrix(W[i,]).transpose(), np.matrix(W[i,]))
        #sig_beta = np.linalg.inv(sig_beta)
        L_beta = np.linalg.cholesky(sig_beta)
        Linv_beta = scipy.linalg.solve_triangular(L_beta, np.eye(J), lower=True)
        sig_beta = np.matmul(Linv_beta.transpose(), Linv_beta)
        
        mu_alp = np.matrix(np.zeros(shape=(L,1))).reshape(-1)
        sum_rx = sum(np.array(R) * np.array((Z - np.matmul(W, beta_y.transpose()) - np.matmul(U,V.transpose()))))#save U'V
        for j in range(m):
            mu_alp = mu_alp + sum_rx[j] * X[j,]
        mu_alp = np.matmul(sig_alp, mu_alp.reshape((L,1)))
        
        mu_beta = np.matrix(np.zeros(shape=(L,1))).reshape(-1)
        sum_rw = sum(np.array(R.transpose()) * np.array(Z.transpose() - np.matmul(X, alpha_y.transpose()) - np.matmul(V,U.transpose())))
        for i in range(n):
            mu_beta = mu_beta + sum_rw[i] * W[i,]
        mu_beta = np.matmul(sig_beta, mu_beta.reshape((L,1)))
        
        #print(mu_alp,sig_alp)
        #alpha_y = np.random.multivariate_normal(mean = np.asarray(mu_alp).reshape(-1), cov=sig_alp, size=1)
        stand_alp = np.random.normal(size=K)
        alpha_y = mu_alp.reshape(-1) + np.matmul(Linv_alp.transpose(),stand_alp.reshape((K,1))).reshape(-1)
        #beta_y = np.random.multivariate_normal(mean = np.asarray(mu_beta).reshape(-1), cov=sig_beta, size=1)
        stand_beta = np.random.normal(size=J)
        beta_y = mu_beta.reshape(-1) + np.matmul(Linv_beta.transpose(),stand_beta.reshape((J,1))).reshape(-1)
        ALP_Y.append(alpha_y)
        BETA_Y.append(beta_y)
        
        # sample U_i and V_j
        for i in range(n):
            sig_ui = np.eye(L) / sigma_u2
            mu_ui = np.matrix(np.zeros(shape=(L,1))).reshape(-1)
            for j in OBi[i]:
                sig_ui = sig_ui + R[i,j] * np.matmul(np.matrix(V[j,]).transpose(), np.matrix(V[j,]))
                mu_ui = mu_ui + R[i,j] * (Z[i,j] - sum(sum(W[i,]*np.array(beta_y))) - sum(sum(X[j,]*np.array(alpha_y)))) * V[j,]
            #sig_ui = np.linalg.inv(sig_ui)
            L_ui = np.linalg.cholesky(sig_ui)
            Linv_ui = scipy.linalg.solve_triangular(L_ui, np.eye(L), lower=True)
            sig_ui = np.matmul(Linv_ui.transpose(), Linv_ui)
            mu_ui = np.matmul(sig_ui, mu_ui.reshape(L,1))
            #U[i,] = np.random.multivariate_normal(mean = np.asarray(mu_ui).reshape(-1), cov=sig_ui, size=1)
            stand_ui = np.random.normal(size=L)
            U[i,] = mu_ui.reshape(-1) + np.matmul(Linv_ui.transpose(),stand_ui.reshape((L,1))).reshape(-1)
        
        for j in range(m):
            sig_vj = np.eye(L) / sigma_v2
            mu_vj = np.matrix(np.zeros(shape=(L,1))).reshape(-1)
            for i in OBj[j]:
                sig_vj = sig_vj + R[i,j] * np.matmul(np.matrix(U[i,]).transpose(), np.matrix(U[i,]))
                mu_vj = mu_vj + R[i,j] * (Z[i,j] - sum(sum(W[i,]*np.array(beta_y))) - sum(sum(X[j,]*np.array(alpha_y)))) * U[i,]
            #sig_vj = np.linalg.inv(sig_vj)
            L_vj = np.linalg.cholesky(sig_vj)
            Linv_vj = scipy.linalg.solve_triangular(L_vj, np.eye(L), lower=True)
            sig_vj = np.matmul(Linv_vj.transpose(), Linv_vj)
            mu_vj = np.matmul(sig_vj, mu_vj.reshape(L,1))
            #V[j,] = np.random.multivariate_normal(mean = np.asarray(mu_vj).reshape(-1), cov=sig_vj, size=1)
            stand_vj = np.random.normal(size=K)
            V[j,] = mu_vj.reshape(-1) + np.matmul(Linv_vj.transpose(),stand_vj.reshape((L,1))).reshape(-1)
        
        UU.append(U)
        VV.append(V)
            
        # sample Z_ij
        zmean = np.matmul(W, beta_y.reshape((J,1))) + np.matmul(X, alpha_y.reshape((K,1))).transpose() + np.matmul(U, V.transpose())
        for i in range(n):
            for j in OBi[i]:
                if Y[i,j] == 1:
                    if stats.norm.cdf(zmean[i,j]) > 0.01:
                        Z[i,j] = stats.truncnorm.rvs(-zmean[i,j], np.inf, loc=zmean[i,j],scale=1,size=1)
                else:
                    if stats.norm.cdf(zmean[i,j]) <0.99:
                        Z[i,j] = stats.truncnorm.rvs(-np.inf, -zmean[i,j], loc=zmean[i,j],scale=1,size=1)
        ZZ.append(Z)
        
        # compute likelihood
        log_like = log_likelihood(Y,R,X,W, Z,U,V,sigma_u2,sigma_v2,alpha_y, beta_y, sig_y2, alp_u, alp_v, beta_u, beta_v)
        LOG_LIKES.append(log_like)
        
        print(alpha_y)
        print(beta_y)

    return SIG_U2, SIG_V2, ALP_Y, BETA_Y, UU, VV, ZZ, LOG_LIKES, TIMES

    
def spec(x, order=2):
    beta, sigma = yule_walker(x,order)
    return sigma**2 / (1. - np.sum(beta))**2

def log_likelihood(Y,R,X,W, Z,U,V,sigma_u2,sigma_v2,alpha_y, beta_y, sig_y2, alp_u, alp_v, beta_u, beta_v):
    n, m = R.shape
    J = W.shape[1]
    K = X.shape[1]
    L = U.shape[1]
    
    ll = 0
    for i in range(n):
        for j in range(m):
            if R[i,j] == 1:
                ll -= 0.5 * (Z[i,j] - sum(sum(W[i,]*np.array(beta_y))) - sum(sum(X[j,]*np.array(alpha_y))) - sum(U[i,]*V[j,]))**2
                ll -= 0.5 * sum(sum(np.array(beta_y)**2))/sig_y2 + 0.5 * sum(sum(np.array(alpha_y)**2))/sig_y2
    ll -= np.log(sigma_u2) * 0.5 * (n*L+2*alp_u+2) + np.log(sigma_v2) * (m*L+2*alp_v+2)*0.5
    ll -= 0.5 * (sum(sum(U**2)) + beta_u) / sigma_u2 + 0.5 * ( sum(sum(V**2)) + beta_v ) / sigma_v2
    
    return(ll[0])

# df is a dataframe of all sampled parameters
def EffectiveSize(df):
    nn, mm = df.shape
    df.columns = ["0"]*mm
    v0 = []
    ESS = []
    for jj in range(mm):
        xx = df.iloc[:,jj]
        xx_mod = AR(xx)
        xx_res = xx_mod.fit(maxlag=100,ic='aic')
        v0.append(xx_res.sigma2/(1.0-sum(xx_res.params))**2)
    for jj in range(mm):
        xx = df.iloc[:,jj]
        ess = xx.std()**2 / v0[jj] * nn
        ESS.append(ess)
    return(ESS)
    
    
    

## Simulations

In [25]:
import os
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.regression.linear_model import yule_walker
import multiprocessing

#dir_name = "/home/yingjin/gibbs/rec_engine_stream/simulations/gibbs/workflow"
#os.chdir(dir_name)
from gibbs_funcs import base_gibbs
from gibbs_funcs import base_gibbs_wrapper
from gibbs_funcs import initial_gibbs
from gibbs_funcs import EffectiveSize

Set Parameters for priors

In [26]:
a_r, b_r = 1, 1 # R_{ij} ~ Bern(p), p ~ Beta(a_r, b_r)
alp_u, beta_u = 1, 1 # U_i ~ N(0, sig_u^2), sig_u^2 ~ IG(alp_u, beta_u)
alp_v, beta_v = 1, 1 # V-j ~ N(0, sig_v^2), sig_v^2 ~ IG(alp_v, beta_v)
sig_y2 = 10 # alpha_y ~ N(0,sigma_y^2 I_K), beta_y ~ N(0,sigma_y^2 I_J)

Set dimensions of data

In [27]:
mnid=1
ns = [100, 1000, 5000]
ms = [20, 200, 2000]
n = ns[mnid]
m = ms[mnid]

In [28]:
random_dim=2
J = random_dim # dimension of beta_y and W_i
K = random_dim # dimension of alpha_y and X_j
L = random_dim # dimension of U_i and V_j
coef_scale=5.0

generate simulation data, where beta0, alpha0 are true parameters

In [29]:
X = np.random.normal(0,1, (m,K))
W = np.random.normal(0,1, (n,J))
beta0 = (2*np.random.binomial(1,0.5,J).reshape((J,1)) -1)* coef_scale #np.random.normal(0,1, (J,1)) # true beta0 ~ uniform, larger value
alpha0 = (2*np.random.binomial(1,0.5,J).reshape((J,1)) -1)* coef_scale #np.random.normal(0,1, (K,1)) # true alpha0
U = np.random.normal(0,1, (n, L)) # truth: sigma_u = 1
V = np.random.normal(0,1, (m, L)) # truth: sigma_v = 1
Zmeans = np.matmul(W, beta0) + np.matmul(X, alpha0).transpose() + np.matmul(U, V.transpose())
Z = Zmeans + np.random.normal(0,1, (n,m)) # var of Zmeans should be large
R = np.random.binomial(1, 0.01, (n,m)) # truth: p_r = 0.1
Y = 1.0 * (Z>0)
for i in range(n):
    for j in range(m):
        if R[i,j]==0:
            Y[i,j] = float("nan")

snr = Zmeans[R>0].std()**2/ Z[R>0].std()**2

In [30]:
snr

0.9853728248444464

In [31]:
N_tot = 50000 # the limit of all iterations
N_sing = 500 # num of iterations before each check
n_chain = 3

## Burn-in Process

In [ ]:
# set parallel computing
pool = multiprocessing.Pool(processes=n_chain)


# initiate
args = []
for i in range(n_chain):
    # initial_gibbs returns tuple (Z,U,V,alpha_y,beta_y,sigma_u2,sigma_v2)
    init = initial_gibbs(Y,W,X,Z,R,L,sig_y2, alp_u, beta_u, alp_v, beta_v, a_r, b_r)
    new_arg = list((N_sing,R,Y,W,X) + init + (alp_u, beta_u, alp_v, beta_v, sig_y2))
    args.append(new_arg)

count = 0

# record results
all_alpy = [pd.DataFrame()] * n_chain
all_betay = [pd.DataFrame()] * n_chain
all_sigu2 = [pd.DataFrame()] * n_chain
all_sigv2 = [pd.DataFrame()] * n_chain
all_paras = [pd.DataFrame()] * n_chain

#########################
# burnin
if_continue = True
while count < N_tot and if_continue:

    # run a batch of N_sing iterations
    results = pool.map(base_gibbs_wrapper, args)
    count += N_sing

    # process the results
    for i in range(n_chain):
        # subtract results for each chain
        tmp_res = results[i]
        tmp_alpy = tmp_res[2]
        tmp_betay = tmp_res[3]
        tmp_sigu2 = tmp_res[0]
        tmp_sigv2 = tmp_res[1]
        UU = tmp_res[4]
        VV = tmp_res[5]
        ZZ = tmp_res[6]

        # update args
        new_arg  = list((N_sing, R,Y,W,X, ZZ[-1], UU[-1], VV[-1],tmp_alpy[-1], tmp_betay[-1], 
                         tmp_sigu2[-1], tmp_sigv2[-1], alp_u, beta_u, alp_v, beta_v, sig_y2))
        args[i] = new_arg

        # concatenate results
        tmp_alpy_mat = np.matrix(np.zeros(shape=(N_sing,J)))
        tmp_betay_mat = np.matrix(np.zeros(shape=(N_sing,J)))

        for jj in range(N_sing):
            tmp_alpy_mat[jj,] = tmp_alpy[jj]
            tmp_betay_mat[jj,] = tmp_betay[jj]

        all_alpy[i] = pd.concat([all_alpy[i], pd.DataFrame(tmp_alpy_mat)]) 
        all_betay[i] = pd.concat([all_betay[i], pd.DataFrame(tmp_betay_mat)]) 
        all_sigu2[i] = pd.concat([all_sigu2[i], pd.DataFrame(tmp_sigu2)]) 
        all_sigv2[i] = pd.concat([all_sigv2[i], pd.DataFrame(tmp_sigu2)]) 

        all_paras[i] = pd.concat([all_paras[i],
                                  pd.concat([pd.DataFrame(tmp_alpy_mat), 
                                             pd.DataFrame(tmp_betay_mat), 
                                             pd.DataFrame(tmp_sigu2), 
                                             pd.DataFrame(tmp_sigv2)],axis=1)])


    # diagnostic for burnin
    # Gelman-Rubin test
    if_pass = True
    n_paras = (all_paras[0]).shape[1]
    NN = int(all_paras[0].shape[0]/2)

    hatrs = np.array([0.0]*n_paras)
    for jj in range(n_paras):
        means = []
        stds = []
        for ii in range(n_chain):
            mean = all_paras[ii].iloc[NN:NN*2,jj].mean()
            sd = all_paras[ii].iloc[NN:NN*2,jj].std()
            means.append(mean)
            stds.append(sd)
        means = np.array(means)
        stds = np.array(stds)
        ww = np.mean(stds**2)
        bb = np.std(means) **2 * NN

        hatvar = (1-1/NN)*ww + bb/NN
        hatr = np.sqrt(hatvar/ww)
        hatrs[jj] = hatr

    if_pass = sum((1.0<=hatrs)&(hatrs<=1.1)) == n_paras
    if if_pass:
        if_continue = False

0
[[ 0.32605869 -0.39142837]]
[[0.08614119 0.28522123]]
0
0
[[ 0.412287   -0.47147704]]
[[ 0.35064057 -0.40623983]]
[[0.21971395 0.34505564]]
[[0.27663687 0.3011075 ]]
1
[[ 0.48765707 -0.55234542]]
[[0.3184873  0.48573055]]
1
[[ 0.50495816 -0.57069893]]
1
[[ 0.54695234 -0.61541201]]
[[0.43750104 0.5002193 ]]
[[0.40497584 0.52840887]]
2
[[ 0.608288   -0.62959339]]
[[0.5156076  0.59154221]]
2
[[ 0.62699706 -0.64729676]]
[[0.5939301  0.61020374]]
2
[[ 0.6584397  -0.67609895]]
[[0.57738784 0.63294554]]
3
[[ 0.7097134  -0.65423728]]
[[0.61763267 0.68792446]]
3
[[ 0.72806579 -0.67426408]]
[[0.67390255 0.70493808]]
3
[[ 0.75149752 -0.69659462]]
[[0.66493241 0.7228721 ]]
4
[[ 0.82647945 -0.72480613]]
[[0.75284805 0.80229364]]
4
[[ 0.8442558  -0.74382923]]
[[0.79568291 0.81930062]]
4
[[ 0.86237065 -0.76073771]]
[[0.79177393 0.83434064]]
5
[[ 0.87888806 -0.8147843 ]]
[[0.8191281  0.84290225]]
5
[[ 0.91129666 -0.84652303]]
5
[[0.85260882 0.87205628]]
[[ 0.89642266 -0.83296856]]
[[0.85348609 0.858

[[1.73292342 1.8391725 ]]
49
[[ 1.84599451 -1.74755029]]
[[1.76933396 1.84639856]]
48
[[ 1.89391688 -1.74567704]]
[[1.73321007 1.86209991]]
48
[[ 1.90052974 -1.75205132]]
[[1.73981306 1.86863683]]
50
[[ 1.84008509 -1.74565212]]
[[1.78290038 1.83683674]]
49
[[ 1.84781577 -1.7492919 ]]
49
[[1.7711319  1.84817779]]
[[ 1.8412827  -1.74302799]]
[[1.76464033 1.84173118]]
51
[[ 1.82884019 -1.79889567]]
[[1.73266683 1.86607154]]
50
[[ 1.84188408 -1.74736099]]
[[1.7846735  1.83858551]]
50
[[ 1.83542861 -1.74121115]]
[[1.7782737  1.83224801]]
51
52
[[ 1.8306088 -1.8005733]]
[[ 1.78458318 -1.85313891]]
[[1.77397291 1.82791928]]
[[1.73441178 1.86779624]]
53
[[ 1.79235686 -1.82785041]]
[[1.77445836 1.85922362]]
51
[[ 1.8242584  -1.79452852]]
52
[[1.72811709 1.86154612]]
[[ 1.78631072 -1.85479884]]
[[1.775677   1.82962327]]
52
[[ 1.78009875 -1.84881616]]
[[1.76952198 1.82345539]]
53
[[ 1.79405236 -1.82948661]]
[[1.77613548 1.86089548]]
54
[[ 1.81324185 -1.83946003]]
[[1.7776333  1.85930427]]
55
[[ 1

[[1.93693578 2.12022271]]
96
[[ 2.12493545 -2.10882136]]
[[1.93334733 2.1166538 ]]
96
[[ 2.12749204 -2.11137625]]
[[1.93594761 2.1192408 ]]
97
[[ 2.15100287 -2.07603159]]
[[1.92350453 2.13688088]]
97
[[ 2.14751275 -2.07255543]]
[[1.9199725  2.13334654]]
97
[[ 2.15004274 -2.07507516]]
[[1.92253183 2.13590847]]
98
[[ 2.14254372 -2.13898667]]
[[1.97065497 2.21390232]]
98
[[ 2.13908702 -2.13553716]]
[[1.96718712 2.21040587]]
99
[[ 2.21049246 -2.11347952]]
[[2.0415822  2.24423255]]
98
[[ 2.14159295 -2.13803748]]
[[1.96970001 2.21294035]]
100
[[ 2.25972978 -2.12091321]]
[[2.03053136 2.20979305]]
99
[[ 2.20708017 -2.11006395]]
[[2.0381732  2.24074937]]
99
[[ 2.2095541  -2.11253966]]
[[2.04064349 2.24327418]]
100
[[ 2.25635266 -2.11754844]]
[[2.02714566 2.2063428 ]]
101
[[ 2.26288    -2.12167549]]
[[2.0435011  2.22626424]]
100
[[ 2.25880123 -2.11998747]]
[[2.02959884 2.20884391]]
101
[[ 2.25951627 -2.11835845]]
[[2.04015341 2.22285969]]
102
[[ 2.20228651 -2.13673761]]
[[2.04701143 2.23532409]]

[[ 2.44598733 -2.46840179]]
[[2.50043335 2.56586895]]
143
[[ 2.21315871 -2.32819247]]
[[2.33377811 2.27054692]]
144
[[ 2.4561438  -2.50292161]]
[[2.44161958 2.5626448 ]]
144
[[ 2.45551958 -2.50231536]]
[[2.44099366 2.56202448]]
144
[[ 2.20337647 -2.34651436]]
[[2.29248608 2.27540472]]
145
[[ 2.42707206 -2.51900451]]
145
[[2.54436777 2.64236976]]
[[ 2.42645544 -2.51840203]]
[[2.5437456  2.64175387]]
145
[[ 2.22299912 -2.34306773]]
[[2.29387083 2.25934078]]
146
[[ 2.449902   -2.52125871]]
[[2.5308163  2.63770894]]
146
[[ 2.4492894  -2.52065863]]
[[2.53020062 2.6370986 ]]
146
[[ 2.25854468 -2.29132418]]
[[2.29741156 2.29434439]]
147
[[ 2.47072746 -2.52017948]]
[[2.55841921 2.61146302]]
147
[[ 2.47133588 -2.52077768]]
[[2.55902648 2.61206758]]
147
[[ 2.24031531 -2.27428243]]
[[2.36500621 2.35249282]]
148
[[ 2.49245335 -2.56707883]]
[[2.52621357 2.54078963]]
148
[[ 2.49305898 -2.56767343]]
[[2.52681696 2.54138624]]
148
[[ 2.31662983 -2.26595169]]
[[2.34594799 2.38368248]]
149
[[ 2.47827272 

[[2.63922839 2.6223703 ]]
191
[[ 2.61930672 -2.69073867]]
[[2.67813176 2.73857962]]
190
[[ 2.5753294  -2.65263789]]
[[2.67335984 2.71178881]]
191
[[ 2.59773115 -2.57405307]]
[[2.66758082 2.62410944]]
191
[[ 2.61973408 -2.69117095]]
[[2.67857312 2.73902357]]
192
[[ 2.58319156 -2.70336123]]
[[2.64800191 2.71723584]]
192
[[ 2.60244354 -2.61757476]]
[[2.64560216 2.65873349]]
192
[[ 2.58361656 -2.70378791]]
[[2.64844133 2.71767709]]
193
[[ 2.56796661 -2.72172187]]
[[2.66295316 2.69945138]]
193
[[ 2.5683895  -2.72214703]]
[[2.6633891  2.69988876]]
193
[[ 2.57179401 -2.56953939]]
[[2.66909493 2.7397082 ]]
194
[[ 2.53675801 -2.73631416]]
[[2.70880935 2.78542915]]
194
[[ 2.53633854 -2.73588914]]
[[2.70837603 2.78499731]]
195
[[ 2.60186487 -2.75276437]]
[[2.70631931 2.78694722]]
194
[[ 2.62106553 -2.59989274]]
[[2.70191614 2.77118801]]
195
[[ 2.60144829 -2.75234229]]
[[2.70588899 2.7865186 ]]
196
[[ 2.63416609 -2.71241823]]
[[2.70058621 2.74816618]]
195
[[ 2.57437175 -2.62010711]]
[[2.72338005 2

...

[[ 3.73119469 -3.77569958]]
[[3.7300166  3.80137792]]
82
[[ 3.88893034 -3.87387988]]
[[4.04765407 3.99458566]]
79
[[ 3.25829791 -3.2621793 ]]
[[3.1720547  3.25489262]]
83
[[ 3.94009543 -3.86497521]]
[[4.02606862 3.98806432]]
78
[[ 3.77296646 -3.83414994]]
[[3.77933475 3.8176446 ]]
80
[[ 3.29142779 -3.24871323]]
[[3.20569715 3.29321242]]
84
[[ 3.9368742  -3.89479577]]
[[3.99315808 4.00428223]]
79
[[ 3.80323196 -3.84092801]]
[[3.76949875 3.77881736]]
81
[[ 3.35773735 -3.2645439 ]]
[[3.2332511  3.31591763]]
85
[[ 3.88165346 -3.9356133 ]]
[[4.01069319 3.99107218]]
80
[[ 3.81353622 -3.79985178]]
[[3.75764845 3.78547617]]
82
[[ 3.34618078 -3.32727009]]
[[3.23060424 3.34985879]]
86
[[ 3.89252546 -3.9157265 ]]
[[3.98070985 3.97595574]]
81
[[ 3.89965056 -3.78333739]]
[[3.77655536 3.81511913]]
83
[[ 3.32199134 -3.25881525]]
[[3.2082109  3.38115837]]
82
[[ 3.88870024 -3.74988303]]
[[3.7191149  3.76947989]]
87
[[ 3.85655452 -3.90030224]]
[[3.9431313  3.99348105]]
84
[[ 3.30353455 -3.23120718]]
[[3

[[3.19703821 3.16257804]]
129
[[ 3.90426004 -3.86252522]]
[[3.85428233 3.87245874]]
125
[[ 3.87717003 -3.79025404]]
[[3.75827473 3.77936034]]
127
[[ 3.25065834 -3.18635622]]
[[3.19602561 3.18715689]]
130
[[ 3.92069751 -3.90478552]]
[[3.84420795 3.84822989]]
126
[[ 3.85877786 -3.82308602]]
[[3.75111696 3.75087038]]
128
[[ 3.29654257 -3.20823647]]
[[3.21360549 3.17651512]]
131
[[ 3.87353664 -3.92662674]]
[[3.8318576  3.90110428]]
129
[[ 3.2994352  -3.22493447]]
[[3.2076005  3.22288027]]
127
[[ 3.8357009 -3.7901756]]
[[3.69804757 3.75494258]]
132
[[ 3.86017721 -3.88475002]]
[[3.83877889 3.92896449]]
130
[[ 3.25471797 -3.25179245]]
[[3.20302238 3.20932725]]
128
[[ 3.82711414 -3.77705202]]
[[3.72470861 3.80288333]]
133
[[ 3.86918578 -3.89812276]]
131
[[ 3.25404546 -3.22935819]]
[[3.91489668 3.96932094]]
[[3.26548056 3.18137885]]
129
[[ 3.82429796 -3.7881246 ]]
[[3.6950113  3.80761717]]
134
[[ 3.86814771 -3.86079553]]
[[3.91619654 3.9574952 ]]
132
[[ 3.29867786 -3.21716237]]
[[3.2501685  3.2

177
[[ 3.83925197 -3.92480402]]
[[3.84214243 4.13967017]]
169
[[ 3.89026974 -3.79917197]]
[[3.82618057 3.87936184]]
176
[[ 3.2913194  -3.19940899]]
[[3.188848   3.25129965]]
178
[[ 3.84391132 -3.89173157]]
[[3.79973581 4.13497107]]
170
[[ 3.8748916  -3.77626489]]
[[3.78291659 3.88706355]]
179
[[ 3.83952315 -3.93687516]]
[[3.75768933 4.13246686]]
177
[[ 3.22979476 -3.29885048]]
[[3.15196991 3.24946237]]
171
[[ 3.90288362 -3.79502782]]
[[3.80808106 3.89643268]]
180
[[ 3.82819782 -3.87600964]]
[[3.72195754 4.17734721]]
178
[[ 3.21288532 -3.25449388]]
[[3.18225215 3.25996118]]
172
[[ 3.90975606 -3.87770856]]
[[3.78891402 3.94575528]]
181
[[ 3.81967755 -3.87748092]]
[[3.68060477 4.17561638]]
179
[[ 3.21775021 -3.23337396]]
[[3.16963688 3.21598507]]
173
[[ 3.93618349 -3.88385802]]
[[3.76422971 3.99522221]]
182
[[ 3.86933017 -3.8406842 ]]
[[3.71287564 4.09891724]]
180
[[ 3.19432437 -3.22714813]]
[[3.17029797 3.21619192]]
174
[[ 3.91657735 -3.91389433]]
[[3.80703352 3.95654573]]
183
[[ 3.86722

[[3.89517331 4.00505243]]
216
[[ 3.97766018 -4.01593069]]
[[3.98185873 4.09258993]]
224
[[ 3.14632404 -3.11863653]]
[[3.0853139  3.27175186]]
217
[[ 4.00692105 -4.00256563]]
[[3.95353958 4.05241599]]
224
[[ 3.99228287 -4.01079168]]
[[3.90020016 4.05642628]]
225
[[ 3.24555683 -3.12709157]]
[[3.03276166 3.2352288 ]]
218
[[ 4.00215241 -3.98421662]]
[[3.96547893 4.08069952]]
225
[[ 4.00108491 -4.02158807]]
[[3.90222214 4.04898274]]
226
[[ 3.26336723 -3.14472321]]
[[3.00045862 3.24285678]]
219
[[ 4.00058009 -3.97686608]]
[[3.94075124 4.0708597 ]]
226
[[ 3.99672929 -4.01335236]]
[[3.95718884 4.06608675]]
227
[[ 3.26724218 -3.12791576]]
[[2.99557904 3.22720488]]
220
[[ 4.01424368 -3.98506156]]
[[3.93507294 4.05980822]]
227
[[ 3.96168656 -4.02060197]]
[[3.95985905 4.11213252]]
228
221
[[ 3.29525183 -3.12987003]]
[[3.03068333 3.21336331]]
[[ 3.99086389 -4.00713356]]
[[3.95939292 4.01119338]]
228
[[ 4.01411344 -4.04130552]]
[[3.98207515 4.10637449]]
222
[[ 4.03989226 -4.04203728]]
[[3.96205304 4

270
264
[[ 3.96959081 -4.03382748]]
[[3.90974642 3.99997121]]
[[ 3.98168373 -3.92218554]]
[[3.93838087 4.13019187]]
271
[[ 3.37510744 -3.26197897]]
[[3.20404851 3.39121316]]
265
[[ 3.96667633 -3.91364259]]
[[3.96052183 4.1474257 ]]
271
[[ 4.02485383 -3.98423563]]
[[3.90600213 3.95372774]]
272
[[ 3.38801336 -3.25837391]]
[[3.2053974 3.3544311]]
266
[[ 4.04811378 -3.84549242]]
[[3.99267256 4.16500628]]
272
[[ 3.98889008 -4.00162813]]
[[3.90022663 4.00919237]]
273
[[ 3.36480266 -3.28331714]]
[[3.21120959 3.34176719]]
267
[[ 4.05298954 -3.87485138]]
[[3.9610537  4.10885232]]
273
[[ 4.01123215 -4.01219977]]
[[3.85340241 4.02489023]]
274
[[ 3.32167615 -3.23324557]]
[[3.2763624  3.33253447]]
268
[[ 4.03620072 -3.89823527]]
[[3.99597231 4.10335867]]
274
[[ 3.99733067 -4.01763162]]
[[3.78345559 4.07122229]]
275
[[ 3.3220408  -3.28077997]]
[[3.36418425 3.37201738]]
269
[[ 4.03583758 -3.89043505]]
[[4.03230245 4.13051756]]
275
[[ 4.02503967 -3.98404099]]
[[3.82066564 4.11952257]]
276
[[ 3.3278754

## After Burn-in: check ESS to stop

In [ ]:
#########################
# after stopping
# put results together
all_parameters = pd.DataFrame()
for i in range(n_chain):
    all_parameters = pd.concat([all_parameters, after_paras[i]])
df_alpy = all_parameters.iloc[:,0:L]
df_betay = all_parameters.iloc[:,L:(2*L)]
df_sigu = all_parameters.iloc[:,2*L]
df_sigv = all_parameters.iloc[:,2*L+1]

df_loglike = pd.DataFrame()
df_times = pd.DataFrame()
for i in range(n_chain):
    df_loglike = pd.concat([df_loglike, after_loglikes[i]])
    df_times = pd.concat([df_times, after_times[i]])
df_snr = pd.DataFrame([snr])


'''
#res = gibbs(N,R,Y,W,X,Z,U,V,alpha_y,beta_y,sigma_u2,sigma_v2,alp_u,beta_u,alp_v,beta_v,sig_y2)


# wrap up results
conv_itrs = int(res[8])

df_sigu = pd.DataFrame(res[0])
df_sigv = pd.DataFrame(res[1])
mt_alpy = np.matrix(np.zeros(shape=(conv_itrs+1,J)))
mt_betay = np.matrix(np.zeros(shape=(conv_itrs+1,J)))
# append truth
mt_alpy[0,] = alpha0.reshape(-1)
mt_betay[0,] = beta0.reshape(-1)

for nn in range(conv_itrs):
    mt_alpy[nn+1,] = res[2][nn]
    mt_betay[nn+1,] = res[3][nn]
df_alpy = pd.DataFrame(mt_alpy)
df_betay = pd.DataFrame(mt_betay)
df_times = pd.DataFrame(res[7])
'''



In [ ]:
# MSE of posterior mean
sigu_err = (df_sigu - 1).mean()[0] #np.sqrt(np.mean(np.power(df_sigu-1,2)))
sigv_err = (df_sigv - 1).mean()[0]
alpy_err = np.sqrt(np.power(df_alpy - alpha0.reshape(-1).mean(),2).mean())
betay_err = np.sqrt(np.power(df_betay - beta0.reshape(-1).mean(),2).mean())
df_res = pd.DataFrame([coef_scale,n,m,L,sigu_err,sigv_err,alpy_err,betay_err])